# Fine Tuning LLM to better understand LTL Semantics

## Install Dependencies

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers" trl peft accelerate bitsandbytes
!pip install protobuf==3.20.3 --force-reinstall --no-deps
!pip install llama-cpp-python --upgrade --no-cache-dir

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8cm6nyxa/unsloth_23c1c3871589406db252ee0b385c8cd4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8cm6nyxa/unsloth_23c1c3871589406db252ee0b385c8cd4
  Resolved https://github.com/unslothai/unsloth.git to commit 79ae18052c939578f303ba2f7823dae03cb00b5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 14.1 MB/s eta 0:0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 278.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 275.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422328 sha256=9d3ea58b26442f0ba7f29f5dd60e76f9c90fe6eeed947aa89004ba10f87c801d
  Stored in directory: /tmp/pip-ephem-wheel-cache-51upat4r/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


## Prepare Dataset

In [ ]:
from datasets import load_dataset
import torch

# Load your prepared dataset from the dataset.jsonl file
dataset = load_dataset("json", data_files="dataset.jsonl", split="train")

print(f"Total samples: {len(dataset)}")

# Format the dataset for the model's chat template
def format_instruction(example):
    """Formats the instruction and output using a clean chat template."""
    instruction = example['instruction']
    output = example['output']

    # Updated system prompt to match your current system
    formatted_text = f"""<|im_start|>system
You are a specialized translator that converts natural language drone commands into Linear Temporal Logic (LTL) formulas.

Available LTL operators:
- F(φ) = eventually φ will be true
- G(φ) = φ is always true
- X(φ) = φ is true in next step
- φ & ψ = both φ and ψ are true
- φ | ψ = either φ or ψ is true
- !φ = φ is not true

Available predicates and actions:
- at(location): drone at specific location
- above(altitude): drone above altitude threshold
- hover(duration): maintain position for seconds
- scan(area): perform sensor sweep
- emergency_return(): return to start position
- land(): controlled landing

Respond only with the LTL formula.
{instruction}
[/INST]
{output}"""

    return {"text": formatted_text}

# Apply the formatting to the entire dataset
dataset = dataset.map(format_instruction)

# Create a train/validation split (e.g., 80/20) for evaluation
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Total samples: 289


Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Training samples: 231
Validation samples: 58


## Load Model & Tokenizer

In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
max_seq_length = 512 # Reduced for shorter LTL outputs

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# A simple, yet effective LoRA configuration for your task
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0, # No dropout for a deterministic translation task
    bias="none",
    use_gradient_checkpointing="unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.9: Fast Mistral patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.9.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Setup & Run SFTTrainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Set a padding token (still required for stable left padding)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Explicitly set padding side to 'left' (still required)
tokenizer.padding_side = "left"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=16,
        learning_rate=2e-4,
        num_train_epochs=6,
        warmup_steps=10,
        eval_strategy="steps",
        eval_steps=20,
        save_strategy="steps",
        save_steps=20,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        save_total_limit=1,
        load_best_model_at_end=False,
        dataloader_drop_last=True,
        remove_unused_columns=True,
        group_by_length=True, # Crucial
    ),
)

print("Starting LTL translation training...")
trainer_stats = trainer.train()

print("Training complete!")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/231 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/58 [00:00<?, ? examples/s]

Starting LTL translation training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 231 | Num Epochs = 6 | Total steps = 90
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: israelavendanojr (israelavendanojr-western-washington-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,0.124100,0.106019
40,0.066900,0.069259
60,0.039800,0.061810
80,0.042300,0.063224


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


## Test Model

In [ ]:
import torch
import transformers
from typing import List # Needed for type hinting in the custom class

# --- CRITICAL FIX: Custom Stopping Criteria Class ---
class StopOnTokens(transformers.StoppingCriteria):
    """
    Custom stopping criteria that stops generation when any token in the
    provided list is generated. Used to enforce a single line output.
    """
    def __init__(self, stop_token_ids: List[int]):
        # Ensure token IDs are unique and converted to a set for fast lookup
        self.stop_token_ids = set(stop_token_ids)

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Check if the last generated token is in the stop set
        # input_ids[0, -1] is the last token ID generated
        last_token_id = input_ids[0, -1].item()
        return last_token_id in self.stop_token_ids
# -----------------------------------------------------

# Assuming 'model' and 'tokenizer' are already loaded and configured
# The tokenizer must be from 'unsloth/mistral-7b-instruct-v0.2-bnb-4bit'

def test_model_performance(model, tokenizer):
    """
    Tests the model's LTL translation ability using the correct Mistral template
    and prevents conversational output using a newline stop sequence.
    """
    print("\nTesting model with modern LTL format...")

    # Define the system instruction exactly as it was used in the training data
    system_prompt = """You are a specialized translator that converts natural language drone commands into Linear Temporal Logic (LTL) formulas.

Available LTL operators:
- F(φ) = eventually φ will be true
- G(φ) = φ is always true
- X(φ) = φ is true in next step
- φ & ψ = both φ and ψ are true
- φ | ψ = either φ or ψ is true
- !φ = φ is not true

Available predicates and actions:
- at(location): drone at specific location
- above(altitude): drone above altitude threshold
- hover(duration): maintain position for seconds
- scan(area): perform sensor sweep
- emergency_return(): return to start position
- land(): controlled landing

Respond only with the LTL formula."""

    test_queries = [
        # ----------------------------------------------------
        # 1. CORE INCLUSION RULES (Verifying minimal, correct LTL)
        # ----------------------------------------------------
        # Baseline Simple Action: F(at)
        "fly to waypoint_c",
        # Simple Persistent Goal: G(above)
        "always fly above 3 meters",
        # Simplify Hover (Rule 2.B.1): F(at & hover)
        "wait at waypoint_x for 25 seconds",
        # Strict Sequencing (Rule 2.B.2): F(A & F(B))
        "check area_2 then return to the pad",
        # Parallel Goals (Rule 2.B.3): F(A & B)
        "go to waypoint_a and scan area_1",
        # Complex Combination: G(A) & F(B)
        "always stay above 1 meter while scanning area_3",
        # Mission Critical Command: F(emergency_return)
        "trigger an emergency return",
        # Complex Sequence: F(A & F(B & F(C)))
        "go to wp_1, then scan area_1, then land",

        # ----------------------------------------------------
        # 2. ERROR HANDLING MANDATE (Verifying correct error tags)
        # ----------------------------------------------------
        # Unknown Location/Parameter (Expected: ERROR(UNKNOWN_LOCATION))
        "navigate to non_existent_location",
        # Vague/Ambigous Command (Expected: ERROR(AMBIGUOUS_QUERY))
        "what is the purpose of the drone",
        # Unsupported Action (Expected: ERROR(UNSUPPORTED_ACTION))
        "do a barrel roll",
        # Irrelevant Query (Expected: ERROR(IRRELEVANT_QUERY))
        "set the time to 5pm",

        # ----------------------------------------------------
        # 3. EXCLUSION LIST VERIFICATION (Verifying removed predicates)
        # ----------------------------------------------------
        # Communication Noise removal (Expected: F(at(wp_alpha)))
        "ensure comms are active while flying to wp_alpha",
        # Obstacle Noise removal (Expected: F(scan(area_beta)))
        "avoid collisions while scanning area_beta",
        # 'Near' Predicate removal (Expected: F(at(waypoint_gamma)))
        "get near waypoint_gamma"
    ]

    # Calculate the stop token ID for the newline character
    stop_token_id = tokenizer.convert_tokens_to_ids("\n")

    # Instantiate the custom stopping criteria
    stop_criteria = StopOnTokens([stop_token_id])

    for query in test_queries:
        # Construct the conversation history required by tokenizer.apply_chat_template
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ]

        # Apply the Mistral template, adding the prompt to start assistant generation
        formatted_input = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False,
                temperature=0.0, # Ensures deterministic output
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                # Use the custom stopping criteria
                stopping_criteria=transformers.StoppingCriteriaList([stop_criteria]),
            )

        # Decode only the generated part of the output
        response_tokens = outputs[0][inputs.input_ids.shape[1]:]
        response = tokenizer.decode(response_tokens, skip_special_tokens=True).strip()

        # Clean up any residual stop tokens or instructions (e.g., [/INST])
        # The custom criteria should handle this, but keep the split for safety
        cleaned_response = response.split('\n')[0].strip()

        print("-" * 50)
        print(f"Query: {query}")
        print(f"Model Output: {cleaned_response}")

# Since you are running this interactively, call the function:
test_model_performance(model, tokenizer)



Testing model with modern LTL format...
--------------------------------------------------
Query: fly to waypoint_c
Model Output: F(at(waypoint_C))
--------------------------------------------------
Query: always fly above 3 meters
Model Output: G(above(3.0))
--------------------------------------------------
Query: wait at waypoint_x for 25 seconds
Model Output: F(at(waypoint_x) & hover(25))
--------------------------------------------------
Query: check area_2 then return to the pad
Model Output: F(scan(area_2) & F(at(landing_pad)))
--------------------------------------------------
Query: go to waypoint_a and scan area_1
Model Output: F(at(waypoint_a) & scan(area_1))
--------------------------------------------------
Query: always stay above 1 meter while scanning area_3
Model Output: G(above(1.0)) & F(scan(area_3))
--------------------------------------------------
Query: trigger an emergency return
Model Output: F(emergency_return())
----------------------------------------------

## Export Fine-Tuned Model

In [ ]:
print("\nSaving models...")

# Save the merged model as a GGUF file for Ollama
print("Saving GGUF model...")
model.save_pretrained_gguf(
    "Llama3_Crazyflie_gguf",
    tokenizer,
    quantization_method="q4_k_m"
)

print("\nAll models saved successfully!")


Saving models...
Saving GGUF model...
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.24 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [04:11<00:00,  7.87s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Llama3_Crazyflie_gguf/pytorch_model.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at Llama3_Crazyflie_gguf into f16 GGUF format.
The output location will be /content/Llama3_Crazyflie_gguf/unsloth.F16.gguf
This might take 3 minutes...


TypeError: Couldn't build proto file into descriptor pool: duplicate file name sentencepiece_model.proto